## Import Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
df = pd.read_excel("CaseStudy_Quality_sample25.xlsx")
df = df.replace(r"^\s*$", np.nan, regex=True)  # Replace empty strings with NaN for easier missing value detection

print("Rows:", len(df))
print("Columns:")
print(df.columns.tolist())
print("Number of columns:", len(df.columns))

df.head()

Rows: 372
Columns:
['timevalue', 'providerkey', 'companynameofficial', 'fiscalperiodend', 'operationstatustype', 'ipostatustype', 'geonameen', 'industrycode', 'REVENUE', 'unit_REVENUE']
Number of columns: 10


,timevalue,providerkey,companynameofficial,fiscalperiodend,operationstatustype,ipostatustype,geonameen,industrycode,REVENUE,unit_REVENUE
0,2019,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP
1,2020,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.355518e+10,GBP
2,2021,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.110757e+10,GBP
3,2022,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,9.359479e+09,GBP
4,2023,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP


### Quality Check 1 — Completeness

This check ensures that all required fields needed for analysis are present.
Missing values may indicate extraction failures or incomplete records.

Some of  the required fields to be checked:
- providerkey
- companynameofficial
- timevalue
- REVENUE
- unit_REVENUE
- geonameen

First lets have a colmun level analysis and see the overview

In [4]:
missing_summary = df.isna().sum().to_frame(name="missing_count")
missing_summary["missing_percent"] = (missing_summary["missing_count"] / len(df) * 100).round(2)

missing_summary.sort_values("missing_percent", ascending=False)

,missing_count,missing_percent
REVENUE,91,24.46
unit_REVENUE,78,20.97
companynameofficial,2,0.54
timevalue,0,0.00
providerkey,0,0.00
fiscalperiodend,0,0.00
ipostatustype,0,0.00
operationstatustype,0,0.00
industrycode,0,0.00
geonameen,0,0.00


Now, lets conduct a row level analysis and mark the missing rows and what they are missing and add it to our df for final export

In [5]:
# Define required columns for data quality check (These are currical for analysis and should not be missing)
required_cols = [
    "providerkey",
    "companynameofficial",
    "timevalue",
    "REVENUE",
    "unit_REVENUE",
    "geonameen"
]

missing_required_mask = df[required_cols].isna()

df["dq_completeness_flag"] = missing_required_mask.any(axis=1)

df["dq_completeness_fields"] = missing_required_mask.apply(
    lambda row: ", ".join(
        [col for col, missing in row.items() if missing]
    ),
    axis=1
)
df.loc[df["dq_completeness_fields"] == "", "dq_completeness_fields"] = np.nan

df[["dq_completeness_flag", "dq_completeness_fields"]].head(15)

,dq_completeness_flag,dq_completeness_fields
0,False,NaN
1,False,NaN
2,False,NaN
3,False,NaN
4,False,NaN
5,False,NaN
6,False,NaN
7,False,NaN
8,False,NaN
9,False,NaN


Two new columns are added to the dataset:

- **dq_completeness_flag**: Indicates whether a row contains missing required values.
- **dq_completeness_fields**: Lists which required fields are missing for that record.

In [6]:
df.head()

,timevalue,providerkey,companynameofficial,fiscalperiodend,operationstatustype,ipostatustype,geonameen,industrycode,REVENUE,unit_REVENUE,dq_completeness_flag,dq_completeness_fields
0,2019,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,False,NaN
1,2020,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.355518e+10,GBP,False,NaN
2,2021,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.110757e+10,GBP,False,NaN
3,2022,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,9.359479e+09,GBP,False,NaN
4,2023,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,False,NaN


### Quality Check 2 — Correctness

#### Purpose
The correctness check verifies whether values follow basic business rules and are plausible. Since no external reference dataset is available, correctness is evaluated using rule-based validation.

#### Rules Used
A record is flagged as incorrect if at least one of the following is true:
- `REVENUE` is negative (revenue should not be below 0)
- `REVENUE` is not numeric
- `unit_REVENUE` is missing while `REVENUE` is present
- `timevalue` should be between 1900 and 2026



In [7]:

# Convert revenue safely to numeric
revenue_num = pd.to_numeric(df["REVENUE"], errors="coerce")

# Rule 1: Negative revenue
rule_negative_revenue = revenue_num.notna() & (revenue_num < 0)

# Rule 2: Non-numeric revenue
rule_nonnumeric_revenue = df["REVENUE"].notna() & revenue_num.isna()

# Rule 3: Revenue present but unit missing
rule_missing_unit = revenue_num.notna() & df["unit_REVENUE"].isna()

# Rule 4: Implausible years
rule_invalid_year = (df["timevalue"] < 1900) | (df["timevalue"] > 2026)

# Combine all rules
df["dq_correctness_flag"] = (
    rule_negative_revenue |
    rule_nonnumeric_revenue |
    rule_missing_unit |
    rule_invalid_year
)

# Reason column
df["dq_correctness_reason"] = ""

df.loc[rule_negative_revenue, "dq_correctness_reason"] += "negative_REVENUE; "
df.loc[rule_nonnumeric_revenue, "dq_correctness_reason"] += "non_numeric_REVENUE; "
df.loc[rule_missing_unit, "dq_correctness_reason"] += "missing_unit_REVENUE; "
df.loc[rule_invalid_year, "dq_correctness_reason"] += "invalid_timevalue; "

# Clean output
df["dq_correctness_reason"] = df["dq_correctness_reason"].str.strip()
df.loc[df["dq_correctness_reason"] == "", "dq_correctness_reason"] = np.nan

# Preview flagged records
df.loc[
    df["dq_correctness_flag"],
    ["providerkey", "timevalue", "REVENUE", "unit_REVENUE", "dq_correctness_reason"]
].head(15)

,providerkey,timevalue,REVENUE,unit_REVENUE,dq_correctness_reason
156,124855,2020,837000000.0,NaN,missing_unit_REVENUE;
244,21STCENMGM.NS,2022,-135208000.0,INR,negative_REVENUE;


In [8]:
df.head()

,timevalue,providerkey,companynameofficial,fiscalperiodend,operationstatustype,ipostatustype,geonameen,industrycode,REVENUE,unit_REVENUE,dq_completeness_flag,dq_completeness_fields,dq_correctness_flag,dq_correctness_reason
0,2019,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,False,NaN,False,NaN
1,2020,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.355518e+10,GBP,False,NaN,False,NaN
2,2021,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.110757e+10,GBP,False,NaN,False,NaN
3,2022,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,9.359479e+09,GBP,False,NaN,False,NaN
4,2023,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,False,NaN,False,NaN


### Quality Check 3 — Consistency

### Purpose
The consistency check verifies whether related records follow stable and logically and format vise consistent  patterns across the dataset. 
### Rules Used

1. **Revenue Unit Consistency**
   
   For the same company (`providerkey`), the revenue unit (`unit_REVENUE`) should remain consistent across all reporting years.  
   A company is flagged if more than one distinct non-missing revenue unit is observed.

2. **Industry Code Mapping Consistency**
   
   The `industrycode` column contains both a numeric industry code and a textual description (e.g., "3290 - Other Manufacturing").  
   Each numeric industry code should correspond to a single consistent description.  
   Records are flagged if the same industry code appears with multiple different descriptions.

3. **Fiscal Period Date Format Consistency** (This is format consistency, not logical consistency.)
   
   The `fiscalperiodend` column should follow a consistent date format ("DD-MMM", e.g., "31-Mar").  
   Records that do not match this expected format are flagged as inconsistent.

4. **Revenue Uniqueness per Company–Year–Industry**

    For each company (`providerkey`), only one revenue value should exist per reporting year (`timevalue`) within the same industry classification (`industrycode`).  
    Records are flagged if multiple revenue values are observed for the same company–year–industry combination, as this may indicate duplicated or inconsistent reporting.





In [9]:
#3.1
units_per_company = (
    df.dropna(subset=["unit_REVENUE"])
      .groupby("providerkey")["unit_REVENUE"]
      .agg(lambda s: sorted(s.unique()))
)

unit_count_per_company = units_per_company.apply(len)
inconsistent_company_unit = unit_count_per_company > 1

df["dq_unit_inconsistent_flag"] = (
    df["providerkey"].map(inconsistent_company_unit).fillna(False).astype(bool)
)

df["dq_unit_inconsistent_units"] = df["providerkey"].map(
    units_per_company.apply(lambda lst: ", ".join(lst))
)

# Clean: keep units only when flagged
df.loc[df["dq_unit_inconsistent_flag"] == False, "dq_unit_inconsistent_units"] = np.nan

# Display
df[df["dq_unit_inconsistent_flag"]][
    ["providerkey", "timevalue", "unit_REVENUE", "dq_unit_inconsistent_units"]
].head(20)

,providerkey,timevalue,unit_REVENUE,dq_unit_inconsistent_units
239,20MICRONS.NS,2023,DKK,"DKK, INR"
240,20MICRONS.NS,2022,INR,"DKK, INR"
241,20MICRONS.NS,2021,INR,"DKK, INR"
242,20MICRONS.NS,2020,INR,"DKK, INR"


In [10]:
# 3.2
# Split industrycode (temporary)
industry_split = (
    df["industrycode"]
    .astype(str)
    .str.split(" - ", n=1, expand=True)
)

industry_split.columns = ["industry_num", "industry_desc"]

# Count descriptions per industry code
desc_counts = (
    industry_split
    .groupby("industry_num")["industry_desc"]
    .nunique()
    .reset_index(name="record_count")
)

# Keep only inconsistent ones
inconsistent_mappings = desc_counts[desc_counts["record_count"] > 1]

# Display result
inconsistent_mappings.head()

,industry_num,record_count


No violations were detected; therefore, no additional flag column was added to the dataset.

In [11]:
#3.3
# Expected format: DD-MMM (e.g., 31-Dec)   
date_pattern = r"^\d{2}-[A-Za-z]{3}$"

df["dq_fiscalperiod_format_issue"] = ~(
    df["fiscalperiodend"].astype(str).str.match(date_pattern)
)

# display results
df[["fiscalperiodend", "dq_fiscalperiod_format_issue"]].head(15)

,fiscalperiodend,dq_fiscalperiod_format_issue
0,31-May,False
1,31-May,False
2,31-May,False
3,31-May,False
4,31-May,False
5,2025-06-30 00:00:00,True
6,2025-06-30 00:00:00,True
7,2025-06-30 00:00:00,True
8,2025-06-30 00:00:00,True
9,2025-06-30 00:00:00,True


In [12]:
#3.4
# Extract industry number 
industry_num = (
    df["industrycode"]
    .astype(str)
    .str.split(" - ", n=1, expand=True)[0]
    .rename("industry_num")
)

# Count records per company-year-industry
record_counts = (
    df.groupby(["providerkey", "timevalue", industry_num])
      .size()
      .reset_index(name="record_count")
)

# Find violations
duplicates = record_counts[record_counts["record_count"] > 1]

# Display result
duplicates.head()

,providerkey,timevalue,industry_num,record_count


No violations were detected; therefore, no additional flag column was added to the dataset.

The following data quality columns are added:

- **dq_unit_inconsistent_flag**  
  Indicates companies using multiple revenue units.

- **dq_unit_inconsistent_units**  
  Lists the distinct revenue units observed for that company.

- **dq_fiscalperiod_format_correct**  
  Indicates records with inconsistent fiscalperod date formats.

In [13]:
df.head()

,timevalue,providerkey,companynameofficial,fiscalperiodend,operationstatustype,ipostatustype,geonameen,industrycode,REVENUE,unit_REVENUE,dq_completeness_flag,dq_completeness_fields,dq_correctness_flag,dq_correctness_reason,dq_unit_inconsistent_flag,dq_unit_inconsistent_units,dq_fiscalperiod_format_issue
0,2019,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,False,NaN,False,NaN,False,NaN,False
1,2020,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.355518e+10,GBP,False,NaN,False,NaN,False,NaN,False
2,2021,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.110757e+10,GBP,False,NaN,False,NaN,False,NaN,False
3,2022,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,9.359479e+09,GBP,False,NaN,False,NaN,False,NaN,False
4,2023,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,False,NaN,False,NaN,False,NaN,False


### LLM-Based Plausibility Validation

A Large Language Model (LLM) is used as an additional data quality layer to evaluate revenue anomalies detected by rule-based checks.

While statistical rules identify unusually large revenue changes, they cannot determine whether these changes reflect real business events or data extraction errors. Therefore, only flagged records are sent to the LLM for contextual evaluation. Maybe values are mathematically not usual but can be reasonable with context.

The model considers company, industry, country, revenue values, and time gaps between observations and returns a structured response containing:
- a plausibility verdict (`Plausible` or `Implausible`)
- a short explanation
- a confidence score



In [14]:
# Ensure numeric revenue exists
df["REVENUE_num"] = pd.to_numeric(df["REVENUE"], errors="coerce")

# sort so shifts / comparisons are correct
df = df.sort_values(["providerkey", "timevalue"]).reset_index(drop=True)

# Pprevious revenue + previous year + gap
df["rev_prev"] = df.groupby("providerkey")["REVENUE_num"].shift(1)
df["prev_year"] = df.groupby("providerkey")["timevalue"].shift(1)
df["year_gap"] = df["timevalue"] - df["prev_year"]  #can be bigger than 1

# change vs last available record 
df["rev_pct_change"] = (df["REVENUE_num"] - df["rev_prev"]) / df["rev_prev"]

# only evaluate when both current & previous exist and previous > 0
valid_pair = df["REVENUE_num"].notna() & df["rev_prev"].notna() & (df["rev_prev"] > 0)

# tuneable thresholds
rule_big_increase = valid_pair & (df["rev_pct_change"] > 2.0)      # > +200%
rule_big_drop     = valid_pair & (df["rev_pct_change"] < -0.70)    # < -70%  # can be changed

# main flag
df["dq_revenue_jump_flag"] = rule_big_increase | rule_big_drop

# reason column 
df["dq_revenue_jump_reason"] = np.select(
    [rule_big_increase, rule_big_drop],
    ["jump_gt_200pct_since_last_record", "drop_gt_70pct_since_last_record"],
    default=None
)

# show results
display(df.loc[df["dq_revenue_jump_flag"],
       ["providerkey","companynameofficial","timevalue","prev_year","year_gap",
        "unit_REVENUE","REVENUE_num","rev_prev","rev_pct_change",
        "dq_revenue_jump_reason"]
      ].head(20)
)
print(f"Total rows: {df['dq_revenue_jump_flag'].sum()}")

,providerkey,companynameofficial,timevalue,prev_year,year_gap,unit_REVENUE,REVENUE_num,rev_prev,rev_pct_change,dq_revenue_jump_reason
14,25526,THE HENDERSON SMALLER COMPANIES INVESTMENT TRU...,2023,2022.0,1.0,GBP,1.003670e+09,2.439000e+08,3.115088,jump_gt_200pct_since_last_record
88,67863,LEEDS GROUP PLC,2022,2021.0,1.0,GBP,2.780000e+06,2.959000e+08,-0.990605,drop_gt_70pct_since_last_record
104,82548,REACH PLC,2023,2022.0,1.0,GBP,5.686000e+09,6.014000e+10,-0.905454,drop_gt_70pct_since_last_record
127,105664,CAFFYNS PUBLIC LIMITED COMPANY,2021,2020.0,1.0,GBP,3.194600e+08,1.650850e+09,-0.806488,drop_gt_70pct_since_last_record
128,105664,CAFFYNS PUBLIC LIMITED COMPANY,2022,2021.0,1.0,GBP,2.514260e+09,3.194600e+08,6.870344,jump_gt_200pct_since_last_record
173,140269,JAMES HALSTEAD PLC,2022,2021.0,1.0,GBP,0.000000e+00,2.663620e+09,-1.000000,drop_gt_70pct_since_last_record
231,21STCENMGM.NS,Twentyfirst Century Management Services Limited,2022,2021.0,1.0,INR,-1.352080e+08,9.974400e+07,-2.355550,drop_gt_70pct_since_last_record
282,AARTIPHARM.NS,Aarti Pharmalabs Limited,2021,2020.0,1.0,INR,1.192034e+10,5.191190e+10,-0.770374,drop_gt_70pct_since_last_record
308,ABB.NS,ABB India Limited,2023,2022.0,1.0,INR,3.967270e+09,8.498410e+10,-0.953318,drop_gt_70pct_since_last_record


Total rows: 9


In [15]:
#LLM plausibility review for flagged revenue jumps
import json
import time
import numpy as np
from openai import OpenAI

def build_prompt(row) -> str:
    company   = str(row.get("companynameofficial", ""))
    industry  = str(row.get("industrycode", ""))
    year      = row.get("timevalue", "")
    prev_year = row.get("prev_year", "")
    gap       = row.get("year_gap", "")
    unit      = str(row.get("unit_REVENUE", ""))
    rev       = row.get("REVENUE_num", None)
    prev      = row.get("rev_prev", None)
    chg       = row.get("rev_pct_change", None)

    return f"""
You are a DATA QUALITY validator reviewing extracted company REVENUE time series.

Primary task:
Evaluate the revenue change using NUMERIC evidence first.
You may optionally use general company or industry knowledge
ONLY if you are confident and it clearly supports the decision.

Rules for using company/industry knowledge:
- Do NOT assert specific real-world events as facts.
- Use general reasoning only (e.g., "cyclical industry", "high-growth sector").
- If unsure, ignore company/industry information.
- If you use it, explicitly mention it in the reason.

Important:
- The change is computed relative to the last available record.
- The year gap can be greater than 1.
- Larger gaps increase uncertainty.

Choose ONE verdict:
- "Plausible": numerically reasonable and no strong signs of unit/scale/extraction issues.
- "Implausible": likely data/unit/scale/extraction problem OR too uncertain to approve.

Strong signals for "Implausible":
- Missing or unclear currency/unit
- Negative revenue values
- Extremely large percentage changes suggesting wrong scale/decimal
- Very large changes combined with multi-year gaps
- Missing or non-numeric revenue values

Strong signals for "Plausible":
- Currency/unit is present and consistent
- Revenue values are positive and numeric
- Percentage change is moderate relative to the year gap
- Large changes are explainable by multi-year gaps (accumulated growth/decline)
- Change magnitude is not indicative of scale or decimal errors
- Current and previous values appear internally consistent

Return STRICT JSON only (no extra text):
{{"verdict":"Plausible|Implausible","reason":"<=20 words","confidence":0-1}}

Data:
Company: {company}
Industry: {industry}
Currency/unit: {unit}
Current year: {year}
Previous record year: {prev_year}
Year gap: {gap}
Revenue current: {rev}
Revenue previous: {prev}
Change since last record (pct): {chg}
""".strip()

#LLM calling function
def llm_plausibility_check(row, model="gpt-5.2", max_retries=2, sleep_s=0.8):  # model selection
    prompt = build_prompt(row)
    last_err = None

    for _ in range(max_retries + 1):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                response_format={"type": "json_object"}  # force JSON
            )

            text = (resp.choices[0].message.content or "").strip()

            # cleanup
            if text.startswith("```"):
                text = text.strip("`")
                text = text.replace("json", "", 1).strip()

            data = json.loads(text)

            verdict = str(data.get("verdict", "")).strip()
            reason = str(data.get("reason", "")).strip()
            conf = data.get("confidence", None)

            if verdict not in {"Plausible", "Implausible"}:
                verdict = "Unknown"

            try:
                conf = float(conf) if conf is not None else None
            except Exception:
                conf = None

            return verdict, reason, conf, text

        except Exception as e:
            last_err = str(e)
            time.sleep(sleep_s)

    return "Error", f"LLM call failed: {last_err}", None, None

In [16]:
# Run LLM only on suspects (9 entires)
flagged_idx = df.index[df["dq_revenue_jump_flag"]].tolist()

# Create result columns
df["dq_llm_verdict"] = pd.Series(index=df.index, dtype="object")
df["dq_llm_reason"] = pd.Series(index=df.index, dtype="object")
df["dq_llm_confidence"] = pd.Series(index=df.index, dtype="float")
df["dq_llm_raw"] = pd.Series(index=df.index, dtype="object")

for i in flagged_idx:
    verdict, reason, conf, raw = llm_plausibility_check(df.loc[i])
    df.at[i, "dq_llm_verdict"] = verdict
    df.at[i, "dq_llm_reason"] = reason
    df.at[i, "dq_llm_confidence"] = conf
    df.at[i, "dq_llm_raw"] = raw

# LLM flag
df["dq_llm_flag"] = (df["dq_llm_verdict"] == "Implausible")

# Show results
df.loc[df["dq_revenue_jump_flag"],
       ["providerkey","companynameofficial","timevalue","prev_year","year_gap",
        "unit_REVENUE","REVENUE_num","rev_prev","rev_pct_change",
        "dq_revenue_jump_reason","dq_llm_verdict","dq_llm_reason","dq_llm_confidence","dq_llm_flag"]
      ]

,providerkey,companynameofficial,timevalue,prev_year,year_gap,unit_REVENUE,REVENUE_num,rev_prev,rev_pct_change,dq_revenue_jump_reason,dq_llm_verdict,dq_llm_reason,dq_llm_confidence,dq_llm_flag
14,25526,THE HENDERSON SMALLER COMPANIES INVESTMENT TRU...,2023,2022.0,1.0,GBP,1.003670e+09,2.439000e+08,3.115088,jump_gt_200pct_since_last_record,Implausible,GBP revenues jump 243.9m to 1.00367bn in 1 yea...,0.78,True
88,67863,LEEDS GROUP PLC,2022,2021.0,1.0,GBP,2.780000e+06,2.959000e+08,-0.990605,drop_gt_70pct_since_last_record,Implausible,Revenue drops from 295.9m to 2.78m GBP in 1 ye...,0.90,True
104,82548,REACH PLC,2023,2022.0,1.0,GBP,5.686000e+09,6.014000e+10,-0.905454,drop_gt_70pct_since_last_record,Implausible,Revenue drops from 60.14bn to 5.686bn GBP in 1...,0.90,True
127,105664,CAFFYNS PUBLIC LIMITED COMPANY,2021,2020.0,1.0,GBP,3.194600e+08,1.650850e+09,-0.806488,drop_gt_70pct_since_last_record,Implausible,Revenue drops 80.6% in one year (GBP 1.65bn to...,0.78,True
128,105664,CAFFYNS PUBLIC LIMITED COMPANY,2022,2021.0,1.0,GBP,2.514260e+09,3.194600e+08,6.870344,jump_gt_200pct_since_last_record,Implausible,GBP revenues jump 319.46m to 2.514bn in 1 year...,0.86,True
173,140269,JAMES HALSTEAD PLC,2022,2021.0,1.0,GBP,0.000000e+00,2.663620e+09,-1.000000,drop_gt_70pct_since_last_record,Implausible,"Revenue drops from 2,663,620,000 GBP to 0 in o...",0.97,True
231,21STCENMGM.NS,Twentyfirst Century Management Services Limited,2022,2021.0,1.0,INR,-1.352080e+08,9.974400e+07,-2.355550,drop_gt_70pct_since_last_record,Implausible,"Revenue is negative in 2022 (-135,208,000 INR)...",0.90,True
282,AARTIPHARM.NS,Aarti Pharmalabs Limited,2021,2020.0,1.0,INR,1.192034e+10,5.191190e+10,-0.770374,drop_gt_70pct_since_last_record,Implausible,Revenue drops 77% in one year (INR 51.9B to 11...,0.78,True
308,ABB.NS,ABB India Limited,2023,2022.0,1.0,INR,3.967270e+09,8.498410e+10,-0.953318,drop_gt_70pct_since_last_record,Implausible,Revenue drops 95.3% in one year; likely unit/s...,0.90,True


In [17]:
df.head()

,timevalue,providerkey,companynameofficial,fiscalperiodend,operationstatustype,ipostatustype,geonameen,industrycode,REVENUE,unit_REVENUE,...,prev_year,year_gap,rev_pct_change,dq_revenue_jump_flag,dq_revenue_jump_reason,dq_llm_verdict,dq_llm_reason,dq_llm_confidence,dq_llm_raw,dq_llm_flag
0,2019,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,...,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False
1,2020,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.355518e+10,GBP,...,2019.0,1.0,-0.193136,False,NaN,NaN,NaN,NaN,NaN,False
2,2021,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.110757e+10,GBP,...,2020.0,1.0,-0.180566,False,NaN,NaN,NaN,NaN,NaN,False
3,2022,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,9.359479e+09,GBP,...,2021.0,1.0,-0.157378,False,NaN,NaN,NaN,NaN,NaN,False
4,2023,19457,PZ CUSSONS PLC,31-May,ACTIVE,PUBLIC,United Kingdom,3290 - Other Manufacturing,1.679984e+10,GBP,...,2022.0,1.0,0.794954,False,NaN,NaN,NaN,NaN,NaN,False


In [18]:
from openpyxl import load_workbook

output_path = "CaseStudy_Quality_sample25_FLAGGED.xlsx"

# Export 
df.to_excel(output_path, index=False, na_rep="N/A")

# load workbook
wb = load_workbook(output_path)
ws = wb.active

# auto-adjust column widths
for column_cells in ws.columns:
    max_length = 0
    column_letter = column_cells[0].column_letter

    for cell in column_cells:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass

    adjusted_width = min(max_length + 2, 50)  # limit max width
    ws.column_dimensions[column_letter].width = adjusted_width

# Save changes
wb.save(output_path)

output_path

'CaseStudy_Quality_sample25_FLAGGED.xlsx'

In [20]:
# summary table
df = pd.read_excel("CaseStudy_Quality_sample25_FLAGGED.xlsx")

def count_true(series):
    return int(series.fillna(False).sum())

measures = [
    ("Completeness", "dq_completeness_flag"),
    ("Correctness", "dq_correctness_flag"),
    ("Consistency", "dq_unit_inconsistent_flag"),
    ("Fiscal Format", "dq_fiscalperiod_format_issue"),
    ("Revenue Plausibility (statistics)", "dq_revenue_jump_flag"),
    ("LLM Desicion", "dq_llm_flag"),
]


summary_rows = []
for measure_name, col in measures:
    flagged_count = count_true(df[col])
    percentage = (flagged_count / len(df)) * 100
    summary_rows.append((measure_name, flagged_count, round(percentage, 2)))

summary_table = pd.DataFrame(summary_rows, columns=["Measure", "Flagged Rows", "Percentage (%)"])
summary_table

,Measure,Flagged Rows,Percentage (%)
0,Completeness,93,25.00
1,Correctness,2,0.54
2,Consistency,4,1.08
3,Fiscal Format,69,18.55
4,Revenue Plausibility (statistics),9,2.42
5,LLM Desicion,9,2.42
